# Avance 4: Modelos Alternativos

**Objetivo:** Explorar, construir, evaluar y comparar al menos seis modelos individuales diferentes para encontrar la configuración óptima que maximice el rendimiento en la tarea de clasificación.

## 1. Configuración del Entorno e Importación de Librerías ⚙️

In [2]:
# Manipulación de datos y utilidades
import pandas as pd
import numpy as np
import joblib
import time
from IPython.display import display

# Preprocesamiento
from sklearn.preprocessing import LabelEncoder, label_binarize

# Modelos (Individuales)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB # Asume características no negativas (ej. TF-IDF)
# from sklearn.naive_bayes import GaussianNB # Alternativa si hay características negativas/continuas
from sklearn.neural_network import MLPClassifier

# Selección y Evaluación
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score

# Google Colab
from google.colab import drive

# Montar Google Drive
drive.mount("/content/drive")

# Ignorar warnings futuros (opcional)
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


## 2. Carga y Preparación de Datos 💾

In [4]:
# Cargar archivos preprocesados
try:
    X_train, X_test, y_train, y_test = joblib.load('/content/drive/MyDrive/Proyecto Integrador/pre_training_data.joblib')
    print("Datos cargados exitosamente.")
    print("Dimensiones X_train:", X_train.shape)
    print("Dimensiones y_train:", y_train.shape)
    print("Dimensiones X_test:", X_test.shape)
    print("Dimensiones y_test:", y_test.shape)
except FileNotFoundError:
    print("Error: El archivo 'pre_training_data.joblib' no se encontró en la ruta especificada.")
    print("Asegúrate de que la ruta '/content/drive/MyDrive/Proyecto Integrador/' sea correcta.")
    # Detener ejecución si no se cargan los datos
    raise SystemExit("Error crítico: Datos no encontrados.")


# Codificar etiquetas para modelos que lo requieran (MNB, MLP)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test) # Usar solo transform en test
class_names = le.classes_ # Guardar nombres de clases originales
print("\nClases originales:", class_names)
print("Primeras 5 etiquetas y_train codificadas:", y_train_encoded[:5])

# Definir métricas para evaluación
scoring = ['accuracy', 'f1_macro', 'roc_auc_ovr']

# Lista para almacenar resultados resumidos de la búsqueda inicial
initial_results_list = []

# Diccionario para guardar los mejores estimadores iniciales
best_estimators_initial = {}

Datos cargados exitosamente.
Dimensiones X_train: (184, 224232)
Dimensiones y_train: (184,)
Dimensiones X_test: (46, 224232)
Dimensiones y_test: (46,)

Clases originales: [-1  0  1]
Primeras 5 etiquetas y_train codificadas: [2 1 1 2 1]


## 3. Entrenamiento y Evaluación Inicial (GridSearch CV=2) 🧪

Se entrenarán 6 modelos individuales diferentes con una búsqueda básica de hiperparámetros (`cv=2`) para una comparación inicial rápida.

### Modelo 1: Regresión Logística

In [5]:
print("--- Entrenando Regresión Logística ---")
model_name = 'Regresión Logística'
clf_lr = LogisticRegression(
    solver="saga",
    max_iter=1000, # Reducido para acelerar, ajustar si no converge
    penalty="l2",
    class_weight="balanced",
    random_state=42
)

param_grid_lr = {
    'C': [0.01, 1, 100] # Grid reducido para rapidez
}

grid_search_lr = GridSearchCV(estimator=clf_lr, param_grid=param_grid_lr, cv=2, scoring=scoring, verbose=1, n_jobs=-1, refit='accuracy')

start_time = time.time()
# La Regresión Logística de sklearn maneja etiquetas string
grid_search_lr.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time

LR_resultados = pd.DataFrame(grid_search_lr.cv_results_)
best_index_lr = grid_search_lr.best_index_

initial_results_list.append({
    'Modelo': model_name,
    'Mejor Accuracy (CV)': grid_search_lr.best_score_,
    'F1 Macro (CV)': LR_resultados.loc[best_index_lr, 'mean_test_f1_macro'],
    'ROC AUC OVR (CV)': LR_resultados.loc[best_index_lr, 'mean_test_roc_auc_ovr'],
    'Tiempo Entrenamiento (s)': training_time,
    'Mejores Parámetros': str(grid_search_lr.best_params_)
})
best_estimators_initial[model_name] = grid_search_lr.best_estimator_

print(f"Mejor Accuracy (CV): {grid_search_lr.best_score_:.4f}")
print(f"Mejores Parámetros: {grid_search_lr.best_params_}")
print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")

--- Entrenando Regresión Logística ---
Fitting 2 folds for each of 3 candidates, totalling 6 fits
Mejor Accuracy (CV): 0.6522
Mejores Parámetros: {'C': 100}
Tiempo de entrenamiento: 779.64 segundos


### Modelo 2: K-Nearest Neighbors (KNN)

In [6]:
print("--- Entrenando K-Nearest Neighbors ---")
model_name = 'KNN'
clf_knn = KNeighborsClassifier()

param_grid_knn = {
    'n_neighbors': [3, 5, 7], # Pocos vecinos para rapidez
    'weights': ['uniform', 'distance']
}

grid_search_knn = GridSearchCV(estimator=clf_knn, param_grid=param_grid_knn, cv=2, scoring=scoring, verbose=1, n_jobs=-1, refit='accuracy')

start_time = time.time()
# KNN de sklearn maneja etiquetas string
grid_search_knn.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time

KNN_resultados = pd.DataFrame(grid_search_knn.cv_results_)
best_index_knn = grid_search_knn.best_index_

initial_results_list.append({
    'Modelo': model_name,
    'Mejor Accuracy (CV)': grid_search_knn.best_score_,
    'F1 Macro (CV)': KNN_resultados.loc[best_index_knn, 'mean_test_f1_macro'],
    'ROC AUC OVR (CV)': KNN_resultados.loc[best_index_knn, 'mean_test_roc_auc_ovr'],
    'Tiempo Entrenamiento (s)': training_time,
    'Mejores Parámetros': str(grid_search_knn.best_params_)
})
best_estimators_initial[model_name] = grid_search_knn.best_estimator_

print(f"Mejor Accuracy (CV): {grid_search_knn.best_score_:.4f}")
print(f"Mejores Parámetros: {grid_search_knn.best_params_}")
print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")

--- Entrenando K-Nearest Neighbors ---
Fitting 2 folds for each of 6 candidates, totalling 12 fits
Mejor Accuracy (CV): 0.6848
Mejores Parámetros: {'n_neighbors': 7, 'weights': 'distance'}
Tiempo de entrenamiento: 9.06 segundos


### Modelo 3: Árbol de Decisión

In [7]:
print("--- Entrenando Árbol de Decisión ---")
model_name = 'Árbol Decisión'
clf_dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')

param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20], # Limitar profundidad para evitar overfitting rápido
    'min_samples_split': [2, 10]
}

grid_search_dt = GridSearchCV(estimator=clf_dt, param_grid=param_grid_dt, cv=2, scoring=scoring, verbose=1, n_jobs=-1, refit='accuracy')

start_time = time.time()
# Decision Tree de sklearn maneja etiquetas string
grid_search_dt.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time

DT_resultados = pd.DataFrame(grid_search_dt.cv_results_)
best_index_dt = grid_search_dt.best_index_

initial_results_list.append({
    'Modelo': model_name,
    'Mejor Accuracy (CV)': grid_search_dt.best_score_,
    'F1 Macro (CV)': DT_resultados.loc[best_index_dt, 'mean_test_f1_macro'],
    'ROC AUC OVR (CV)': DT_resultados.loc[best_index_dt, 'mean_test_roc_auc_ovr'],
    'Tiempo Entrenamiento (s)': training_time,
    'Mejores Parámetros': str(grid_search_dt.best_params_)
})
best_estimators_initial[model_name] = grid_search_dt.best_estimator_

print(f"Mejor Accuracy (CV): {grid_search_dt.best_score_:.4f}")
print(f"Mejores Parámetros: {grid_search_dt.best_params_}")
print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")

--- Entrenando Árbol de Decisión ---
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Mejor Accuracy (CV): 0.5707
Mejores Parámetros: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2}
Tiempo de entrenamiento: 48.76 segundos


### Modelo 4: Support Vector Classifier (SVC)

In [8]:
print("--- Entrenando Support Vector Classifier ---")
model_name = 'SVC'
clf_svc = SVC(
    random_state=42,
    class_weight='balanced',
    probability=True # Necesario para ROC AUC
)

param_grid_svc = {
    'C': [0.1, 1, 10], # Grid reducido
    'gamma': [1, 'scale'], # Simplificado
    'kernel': ['rbf'] # Enfocarse en RBF inicialmente
}

grid_search_svc = GridSearchCV(estimator=clf_svc, param_grid=param_grid_svc, cv=2, scoring=scoring, verbose=1, n_jobs=-1, refit='accuracy')

start_time = time.time()
# SVC de sklearn maneja etiquetas string
grid_search_svc.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time

SVM_resultados = pd.DataFrame(grid_search_svc.cv_results_)
best_index_svc = grid_search_svc.best_index_

initial_results_list.append({
    'Modelo': model_name,
    'Mejor Accuracy (CV)': grid_search_svc.best_score_,
    'F1 Macro (CV)': SVM_resultados.loc[best_index_svc, 'mean_test_f1_macro'],
    'ROC AUC OVR (CV)': SVM_resultados.loc[best_index_svc, 'mean_test_roc_auc_ovr'],
    'Tiempo Entrenamiento (s)': training_time,
    'Mejores Parámetros': str(grid_search_svc.best_params_)
})
best_estimators_initial[model_name] = grid_search_svc.best_estimator_

print(f"Mejor Accuracy (CV): {grid_search_svc.best_score_:.4f}")
print(f"Mejores Parámetros: {grid_search_svc.best_params_}")
print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")

--- Entrenando Support Vector Classifier ---
Fitting 2 folds for each of 6 candidates, totalling 12 fits
Mejor Accuracy (CV): 0.6848
Mejores Parámetros: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Tiempo de entrenamiento: 155.23 segundos


### Modelo 5: Multinomial Naive Bayes (MNB)
*Nota: Este modelo asume que las características de entrada son no negativas (ej., conteos de palabras, TF-IDF). Si tus datos preprocesados tienen valores negativos (ej., por escalado StandardScaler), este modelo podría fallar o dar resultados incorrectos. Considera usar `GaussianNB` como alternativa si es el caso.*

In [11]:
import scipy.sparse # Add this import if not already present

print("--- Entrenando Multinomial Naive Bayes ---")
model_name = 'Naive Bayes MNB'

# Safer check for negative values in sparse matrix
negative_check = False
if scipy.sparse.issparse(X_train):
    # For sparse matrices, check the minimum recorded value
    if X_train.nnz > 0: # Ensure the matrix is not empty
       negative_check = X_train.min() < 0
else:
    # For dense arrays, np.any is fine
    negative_check = np.any(X_train < 0)


if negative_check:
    print("ADVERTENCIA: X_train contiene valores negativos. MultinomialNB no es apropiado. Considera usar GaussianNB.")
    # You might want to stop or switch model here depending on requirements
    # For now, we proceed but MNB results might be unreliable
    clf_nb = MultinomialNB()
    param_grid_nb = {
      'alpha': [0.1, 1.0, 10.0]
    }
else:
    print("X_train no contiene valores negativos. Procediendo con MultinomialNB.")
    clf_nb = MultinomialNB()
    param_grid_nb = {
      'alpha': [0.1, 1.0, 10.0] # Probar pocos valores de suavizado
    }


grid_search_nb = GridSearchCV(estimator=clf_nb, param_grid=param_grid_nb, cv=2, scoring=scoring, verbose=1, n_jobs=-1, refit='accuracy')

start_time = time.time()
# MNB generalmente funciona mejor con etiquetas numéricas, usamos las codificadas
grid_search_nb.fit(X_train, y_train_encoded)
end_time = time.time()
training_time = end_time - start_time

MNB_resultados = pd.DataFrame(grid_search_nb.cv_results_)
best_index_nb = grid_search_nb.best_index_

initial_results_list.append({
    'Modelo': model_name,
    'Mejor Accuracy (CV)': grid_search_nb.best_score_,
    'F1 Macro (CV)': MNB_resultados.loc[best_index_nb, 'mean_test_f1_macro'],
    'ROC AUC OVR (CV)': MNB_resultados.loc[best_index_nb, 'mean_test_roc_auc_ovr'],
    'Tiempo Entrenamiento (s)': training_time,
    'Mejores Parámetros': str(grid_search_nb.best_params_)
})
best_estimators_initial[model_name] = grid_search_nb.best_estimator_

print(f"Mejor Accuracy (CV): {grid_search_nb.best_score_:.4f}")
print(f"Mejores Parámetros: {grid_search_nb.best_params_}")
print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")

--- Entrenando Multinomial Naive Bayes ---
X_train no contiene valores negativos. Procediendo con MultinomialNB.
Fitting 2 folds for each of 3 candidates, totalling 6 fits
Mejor Accuracy (CV): 0.6739
Mejores Parámetros: {'alpha': 0.1}
Tiempo de entrenamiento: 1.02 segundos


### Modelo 6: Multi-layer Perceptron (MLP)

In [12]:
print("--- Entrenando Multi-layer Perceptron ---")
model_name = 'Red Neuronal MLP'
clf_mlp = MLPClassifier(
    random_state=42,
    max_iter=100, # Reducido para acelerar
    early_stopping=True # Usar early stopping para evitar overfitting y acelerar
)

param_grid_mlp = {
    'hidden_layer_sizes': [(16,), (32,), (16, 8)], # Arquitecturas pequeñas/medianas
    'alpha': [0.001, 0.01], # Regularización
    'learning_rate_init': [0.001, 0.01]
}

grid_search_mlp = GridSearchCV(estimator=clf_mlp, param_grid=param_grid_mlp, cv=2, scoring=scoring, verbose=1, n_jobs=-1, refit='accuracy')

start_time = time.time()
# MLP requiere etiquetas numéricas
grid_search_mlp.fit(X_train, y_train_encoded)
end_time = time.time()
training_time = end_time - start_time

MLP_resultados = pd.DataFrame(grid_search_mlp.cv_results_)
best_index_mlp = grid_search_mlp.best_index_

initial_results_list.append({
    'Modelo': model_name,
    'Mejor Accuracy (CV)': grid_search_mlp.best_score_,
    'F1 Macro (CV)': MLP_resultados.loc[best_index_mlp, 'mean_test_f1_macro'],
    'ROC AUC OVR (CV)': MLP_resultados.loc[best_index_mlp, 'mean_test_roc_auc_ovr'],
    'Tiempo Entrenamiento (s)': training_time,
    'Mejores Parámetros': str(grid_search_mlp.best_params_)
})
best_estimators_initial[model_name] = grid_search_mlp.best_estimator_

print(f"Mejor Accuracy (CV): {grid_search_mlp.best_score_:.4f}")
print(f"Mejores Parámetros: {grid_search_mlp.best_params_}")
print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")

--- Entrenando Multi-layer Perceptron ---
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Mejor Accuracy (CV): 0.6793
Mejores Parámetros: {'alpha': 0.001, 'hidden_layer_sizes': (16,), 'learning_rate_init': 0.001}
Tiempo de entrenamiento: 112.00 segundos


## 4. Tabla Comparativa y Selección de los Mejores Modelos 📊

In [13]:
tabla_comparativa = pd.DataFrame(initial_results_list)
tabla_comparativa = tabla_comparativa.sort_values(by='Mejor Accuracy (CV)', ascending=False).reset_index(drop=True)

print("\n### Tabla Comparativa de Modelos (Resultados del GridSearchCV Inicial)")
display(tabla_comparativa[['Modelo', 'Mejor Accuracy (CV)', 'F1 Macro (CV)', 'ROC AUC OVR (CV)', 'Tiempo Entrenamiento (s)', 'Mejores Parámetros']])

# Seleccionar los dos mejores modelos basados en Accuracy
top_2_model_names = tabla_comparativa.loc[0:1, 'Modelo'].tolist()

# Recuperar los objetos estimadores correspondientes
clf_top1_initial = best_estimators_initial[top_2_model_names[0]]
clf_top2_initial = best_estimators_initial[top_2_model_names[1]]

# Guardar información necesaria para el ajuste fino
top_models_config = {}
for model_name in top_2_model_names:
    # Asume que MNB y MLP necesitan encoding, los otros no. Ajustar si usaste GaussianNB u otros.
    needs_encoding = model_name in ['Naive Bayes MNB', 'Red Neuronal MLP']
    top_models_config[model_name] = {
        'estimator_class': best_estimators_initial[model_name].__class__,
        'needs_encoding': needs_encoding
    }

print(f"\nLos dos mejores modelos seleccionados para ajuste fino son: {top_2_model_names[0]} y {top_2_model_names[1]}")


### Tabla Comparativa de Modelos (Resultados del GridSearchCV Inicial)


,Modelo,Mejor Accuracy (CV),F1 Macro (CV),ROC AUC OVR (CV),Tiempo Entrenamiento (s),Mejores Parámetros
0,KNN,0.684783,0.410817,0.563294,9.057672,"{'n_neighbors': 7, 'weights': 'distance'}"
1,SVC,0.684783,0.306152,0.468413,155.234080,"{'C': 1, 'gamma': 1, 'kernel': 'rbf'}"
2,Red Neuronal MLP,0.679348,0.310515,0.634666,111.997041,"{'alpha': 0.001, 'hidden_layer_sizes': (16,), ..."
3,Naive Bayes MNB,0.673913,0.268398,0.659207,1.015642,{'alpha': 0.1}
4,Regresión Logística,0.652174,0.377232,0.623814,779.635991,{'C': 100}
5,Árbol Decisión,0.570652,0.425589,0.561060,48.764184,"{'criterion': 'gini', 'max_depth': None, 'min_..."



Los dos mejores modelos seleccionados para ajuste fino son: KNN y SVC


## 5. Ajuste Fino (Fine-Tuning) de los Dos Mejores Modelos 🛠️

Se realizará una búsqueda de hiperparámetros más exhaustiva (`cv=5`) para los dos modelos seleccionados.

In [14]:
model_name_1 = top_2_model_names[0]
print(f"\n--- Ajuste Fino: {model_name_1} ---")

# Re-instanciar con parámetros base y definir grid para ajuste fino
EstimatorClass1 = top_models_config[model_name_1]['estimator_class']
needs_encoding1 = top_models_config[model_name_1]['needs_encoding']
# Asegurar que se pasen los parámetros correctos al instanciar
if 'random_state' in EstimatorClass1().get_params():
    clf_top1 = EstimatorClass1(random_state=42)
else:
     clf_top1 = EstimatorClass1()

# Definir grids de ajuste fino (EJEMPLOS, AJUSTAR SEGÚN EL MODELO SELECCIONADO)
param_grid_fine_tuning_1 = {}
if model_name_1 == 'Regresión Logística':
    clf_top1 = LogisticRegression(solver="saga", max_iter=2000, penalty="l2", class_weight="balanced", random_state=42)
    param_grid_fine_tuning_1 = {'C': [0.5, 1, 5, 10, 50, 100]}
elif model_name_1 == 'KNN':
    param_grid_fine_tuning_1 = {'n_neighbors': [3, 5, 7, 9, 11, 15], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}
elif model_name_1 == 'Árbol Decisión':
    clf_top1 = DecisionTreeClassifier(random_state=42, class_weight='balanced')
    param_grid_fine_tuning_1 = {'criterion': ['gini', 'entropy'], 'max_depth': [5, 10, 15, 20, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]}
elif model_name_1 == 'SVC':
    clf_top1 = SVC(random_state=42, class_weight='balanced', probability=True)
    param_grid_fine_tuning_1 = {'C': [0.5, 1, 5, 10], 'gamma': [0.1, 1, 'scale'], 'kernel': ['rbf']} # Ajustar gamma si scale fue el mejor
elif model_name_1 == 'Naive Bayes MNB':
    param_grid_fine_tuning_1 = {'alpha': np.logspace(-3, 1, 5)} # Ej: [0.001, 0.01, 0.1, 1., 10.]
elif model_name_1 == 'Red Neuronal MLP':
     clf_top1 = MLPClassifier(random_state=42, max_iter=150, early_stopping=True, n_iter_no_change=10)
     param_grid_fine_tuning_1 = {'hidden_layer_sizes': [(16,), (32,), (64,), (32, 16)], 'alpha': [0.0001, 0.001, 0.01], 'learning_rate_init': [0.001, 0.005, 0.01]}

grid_search_fine_tuning_1 = GridSearchCV(estimator=clf_top1,
                                         param_grid=param_grid_fine_tuning_1,
                                         cv=5, # Usar 5 folds
                                         scoring=scoring,
                                         verbose=2,
                                         n_jobs=-1,
                                         refit='accuracy')

y_train_target_1 = y_train_encoded if needs_encoding1 else y_train
start_time_ft1 = time.time()
grid_search_fine_tuning_1.fit(X_train, y_train_target_1)
end_time_ft1 = time.time()
tuning_time_1 = end_time_ft1 - start_time_ft1

print(f"\nResultados Ajuste Fino para {model_name_1}:")
print(f"Mejores Parámetros: {grid_search_fine_tuning_1.best_params_}")
print(f"Mejor Accuracy (CV=5): {grid_search_fine_tuning_1.best_score_:.4f}")
print(f"Tiempo de ajuste fino: {tuning_time_1:.2f} segundos")
best_model_tuned_1 = grid_search_fine_tuning_1.best_estimator_


--- Ajuste Fino: KNN ---
Fitting 5 folds for each of 24 candidates, totalling 120 fits

Resultados Ajuste Fino para KNN:
Mejores Parámetros: {'metric': 'euclidean', 'n_neighbors': 15, 'weights': 'distance'}
Mejor Accuracy (CV=5): 0.6742
Tiempo de ajuste fino: 119.83 segundos


In [15]:
model_name_2 = top_2_model_names[1]
print(f"\n--- Ajuste Fino: {model_name_2} ---")

# Re-instanciar con parámetros base y definir grid para ajuste fino
EstimatorClass2 = top_models_config[model_name_2]['estimator_class']
needs_encoding2 = top_models_config[model_name_2]['needs_encoding']
if 'random_state' in EstimatorClass2().get_params():
    clf_top2 = EstimatorClass2(random_state=42)
else:
     clf_top2 = EstimatorClass2()


# Definir grids de ajuste fino (EJEMPLOS, AJUSTAR SEGÚN EL MODELO SELECCIONADO)
param_grid_fine_tuning_2 = {}
if model_name_2 == 'Regresión Logística':
    clf_top2 = LogisticRegression(solver="saga", max_iter=2000, penalty="l2", class_weight="balanced", random_state=42)
    param_grid_fine_tuning_2 = {'C': [0.5, 1, 5, 10, 50, 100]}
elif model_name_2 == 'KNN':
    param_grid_fine_tuning_2 = {'n_neighbors': [3, 5, 7, 9, 11, 15], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}
elif model_name_2 == 'Árbol Decisión':
    clf_top2 = DecisionTreeClassifier(random_state=42, class_weight='balanced')
    param_grid_fine_tuning_2 = {'criterion': ['gini', 'entropy'], 'max_depth': [5, 10, 15, 20, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]}
elif model_name_2 == 'SVC':
    clf_top2 = SVC(random_state=42, class_weight='balanced', probability=True)
    param_grid_fine_tuning_2 = {'C': [0.5, 1, 5, 10], 'gamma': [0.1, 1, 'scale'], 'kernel': ['rbf']}
elif model_name_2 == 'Naive Bayes MNB':
     param_grid_fine_tuning_2 = {'alpha': np.logspace(-3, 1, 5)} # Ej: [0.001, 0.01, 0.1, 1., 10.]
elif model_name_2 == 'Red Neuronal MLP':
     clf_top2 = MLPClassifier(random_state=42, max_iter=150, early_stopping=True, n_iter_no_change=10)
     param_grid_fine_tuning_2 = {'hidden_layer_sizes': [(16,), (32,), (64,), (32, 16)], 'alpha': [0.0001, 0.001, 0.01], 'learning_rate_init': [0.001, 0.005, 0.01]}


grid_search_fine_tuning_2 = GridSearchCV(estimator=clf_top2,
                                         param_grid=param_grid_fine_tuning_2,
                                         cv=5, # Usar 5 folds
                                         scoring=scoring,
                                         verbose=2,
                                         n_jobs=-1,
                                         refit='accuracy')

y_train_target_2 = y_train_encoded if needs_encoding2 else y_train
start_time_ft2 = time.time()
grid_search_fine_tuning_2.fit(X_train, y_train_target_2)
end_time_ft2 = time.time()
tuning_time_2 = end_time_ft2 - start_time_ft2

print(f"\nResultados Ajuste Fino para {model_name_2}:")
print(f"Mejores Parámetros: {grid_search_fine_tuning_2.best_params_}")
print(f"Mejor Accuracy (CV=5): {grid_search_fine_tuning_2.best_score_:.4f}")
print(f"Tiempo de ajuste fino: {tuning_time_2:.2f} segundos")
best_model_tuned_2 = grid_search_fine_tuning_2.best_estimator_


--- Ajuste Fino: SVC ---
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Resultados Ajuste Fino para SVC:
Mejores Parámetros: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Mejor Accuracy (CV=5): 0.6848
Tiempo de ajuste fino: 1187.74 segundos


## 6. Evaluación Final en Conjunto de Prueba ✅

Evaluamos los dos modelos afinados con los datos de `X_test` y `y_test`.

In [17]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
import numpy as np # Ensure numpy is imported

results_test = {}

# Convert class_names to string array *if* they are not already strings
# This handles cases where original labels might have been numeric
if 'le' in globals() and hasattr(le, 'classes_'):
    if not isinstance(le.classes_[0], str):
        class_names_str = le.classes_.astype(str)
    else:
        class_names_str = le.classes_
else:
    # Fallback if LabelEncoder wasn't used properly or class names unavailable
    unique_labels = np.unique(y_train)
    class_names_str = unique_labels.astype(str)


# --- Evaluación Modelo 1 Afinado ---
model_name_1 = top_2_model_names[0] # Get name from previous cell
needs_encoding1 = top_models_config[model_name_1]['needs_encoding'] # Get encoding flag

print(f"\n--- Evaluación en Test: {model_name_1} --- ")
y_test_target_1 = y_test_encoded if needs_encoding1 else y_test
y_pred_tuned_1 = best_model_tuned_1.predict(X_test)

# Calcular métricas básicas
accuracy_tuned_1 = accuracy_score(y_test_target_1, y_pred_tuned_1)
f1_macro_tuned_1 = f1_score(y_test_target_1, y_pred_tuned_1, average='macro')

# CORRECTED: Always pass string names if not using encoded labels
report_tuned_1 = classification_report(y_test_target_1, y_pred_tuned_1, target_names=class_names_str if not needs_encoding1 else None, digits=4)

# Calcular ROC AUC (maneja error si predict_proba no está disponible)
roc_auc_tuned_1 = np.nan # Valor por defecto
if hasattr(best_model_tuned_1, "predict_proba"):
    try:
        y_proba_tuned_1 = best_model_tuned_1.predict_proba(X_test)
        # Binarizar y_test_target para ROC AUC multiclase
        unique_test_labels_1 = np.unique(y_test_target_1)
        y_test_binarized_1 = label_binarize(y_test_target_1, classes=unique_test_labels_1)

        # Handle binary vs multiclass for roc_auc_score input shapes
        if len(unique_test_labels_1) == 2:
             # Use probability of the positive class (usually class 1)
             roc_auc_tuned_1 = roc_auc_score(y_test_target_1, y_proba_tuned_1[:, 1])
        elif y_test_binarized_1.shape[1] == y_proba_tuned_1.shape[1]:
             roc_auc_tuned_1 = roc_auc_score(y_test_binarized_1, y_proba_tuned_1, multi_class='ovr', average='macro')
        else:
            print(f"Advertencia: No se pudo calcular ROC AUC para {model_name_1} debido a inconsistencia de dimensiones.")

    except Exception as e:
        print(f"Advertencia: No se pudo calcular predict_proba o ROC AUC para {model_name_1}. Error: {e}")
else:
    print(f"Advertencia: El modelo {model_name_1} no tiene el método predict_proba, no se calculará ROC AUC.")


print(f"Accuracy en Test: {accuracy_tuned_1:.4f}")
print("Classification Report en Test:\n", report_tuned_1)
print(f"F1 Macro en Test: {f1_macro_tuned_1:.4f}")
print(f"ROC AUC (OVR, Macro) en Test: {roc_auc_tuned_1:.4f}" if not np.isnan(roc_auc_tuned_1) else "ROC AUC (OVR, Macro) en Test: No calculado")

results_test[model_name_1] = {
    'Accuracy': accuracy_tuned_1,
    'F1 Macro': f1_macro_tuned_1,
    'ROC AUC OVR': roc_auc_tuned_1
}

# --- Evaluación Modelo 2 Afinado ---
model_name_2 = top_2_model_names[1] # Get name from previous cell
needs_encoding2 = top_models_config[model_name_2]['needs_encoding'] # Get encoding flag

print(f"\n--- Evaluación en Test: {model_name_2} --- ")
y_test_target_2 = y_test_encoded if needs_encoding2 else y_test
y_pred_tuned_2 = best_model_tuned_2.predict(X_test)

# Calcular métricas básicas
accuracy_tuned_2 = accuracy_score(y_test_target_2, y_pred_tuned_2)
f1_macro_tuned_2 = f1_score(y_test_target_2, y_pred_tuned_2, average='macro')

# CORRECTED: Always pass string names if not using encoded labels
report_tuned_2 = classification_report(y_test_target_2, y_pred_tuned_2, target_names=class_names_str if not needs_encoding2 else None, digits=4)

# Calcular ROC AUC
roc_auc_tuned_2 = np.nan
if hasattr(best_model_tuned_2, "predict_proba"):
    try:
        y_proba_tuned_2 = best_model_tuned_2.predict_proba(X_test)
        unique_test_labels_2 = np.unique(y_test_target_2)
        y_test_binarized_2 = label_binarize(y_test_target_2, classes=unique_test_labels_2)

        if len(unique_test_labels_2) == 2:
             roc_auc_tuned_2 = roc_auc_score(y_test_target_2, y_proba_tuned_2[:, 1])
        elif y_test_binarized_2.shape[1] == y_proba_tuned_2.shape[1]:
             roc_auc_tuned_2 = roc_auc_score(y_test_binarized_2, y_proba_tuned_2, multi_class='ovr', average='macro')
        else:
             print(f"Advertencia: No se pudo calcular ROC AUC para {model_name_2} debido a inconsistencia de dimensiones.")
    except Exception as e:
        print(f"Advertencia: No se pudo calcular predict_proba o ROC AUC para {model_name_2}. Error: {e}")
else:
     print(f"Advertencia: El modelo {model_name_2} no tiene el método predict_proba, no se calculará ROC AUC.")


print(f"Accuracy en Test: {accuracy_tuned_2:.4f}")
print("Classification Report en Test:\n", report_tuned_2)
print(f"F1 Macro en Test: {f1_macro_tuned_2:.4f}")
print(f"ROC AUC (OVR, Macro) en Test: {roc_auc_tuned_2:.4f}" if not np.isnan(roc_auc_tuned_2) else "ROC AUC (OVR, Macro) en Test: No calculado")

results_test[model_name_2] = {
    'Accuracy': accuracy_tuned_2,
    'F1 Macro': f1_macro_tuned_2,
    'ROC AUC OVR': roc_auc_tuned_2
}


--- Evaluación en Test: KNN --- 
Accuracy en Test: 0.6522
Classification Report en Test:
               precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         7
           0     0.6818    0.9677    0.8000        31
           1     0.0000    0.0000    0.0000         8

    accuracy                         0.6522        46
   macro avg     0.2273    0.3226    0.2667        46
weighted avg     0.4595    0.6522    0.5391        46

F1 Macro en Test: 0.2667
ROC AUC (OVR, Macro) en Test: 0.6196

--- Evaluación en Test: SVC --- 
Accuracy en Test: 0.6522
Classification Report en Test:
               precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         7
           0     0.6818    0.9677    0.8000        31
           1     0.0000    0.0000    0.0000         8

    accuracy                         0.6522        46
   macro avg     0.2273    0.3226    0.2667        46
weighted avg     0.4595    0.6522    0.5391      

## 7. Elección del Modelo Final y Justificación 💡

In [18]:
# Comparar resultados en Test para determinar el modelo final (basado en Accuracy)
acc1 = results_test[model_name_1]['Accuracy']
acc2 = results_test[model_name_2]['Accuracy']

if acc1 >= acc2:
    final_model_name = model_name_1
    other_model_name = model_name_2
    final_accuracy = acc1
    other_accuracy = acc2
    final_f1 = results_test[model_name_1]['F1 Macro']
    other_f1 = results_test[model_name_2]['F1 Macro']
    final_roc_auc = results_test[model_name_1]['ROC AUC OVR']
    other_roc_auc = results_test[model_name_2]['ROC AUC OVR']
else:
    final_model_name = model_name_2
    other_model_name = model_name_1
    final_accuracy = acc2
    other_accuracy = acc1
    final_f1 = results_test[model_name_2]['F1 Macro']
    other_f1 = results_test[model_name_1]['F1 Macro']
    final_roc_auc = results_test[model_name_2]['ROC AUC OVR']
    other_roc_auc = results_test[model_name_1]['ROC AUC OVR']

# Recuperar tiempos de entrenamiento iniciales para la justificación
time_final = tabla_comparativa[tabla_comparativa['Modelo'] == final_model_name]['Tiempo Entrenamiento (s)'].iloc[0]
time_other = tabla_comparativa[tabla_comparativa['Modelo'] == other_model_name]['Tiempo Entrenamiento (s)'].iloc[0]

print(f"Modelo Final Seleccionado (basado en Accuracy en Test): {final_model_name}")

Modelo Final Seleccionado (basado en Accuracy en Test): KNN


### Justificación

Después de realizar el ajuste fino (con `cv=5`) y evaluar los dos mejores modelos identificados en la fase inicial (KNN y SVC) en el conjunto de prueba, los resultados finales son:

* **KNN (Afinado):**
    * Accuracy (Test): 0.6522
    * F1-Macro (Test): 0.2667
    * ROC AUC OVR (Test): 0.6196
    * Tiempo Entrenamiento Inicial (GridSearchCV CV=2): 9.06 segundos

* **SVC (Afinado):**
    * Accuracy (Test): 0.6522
    * F1-Macro (Test): 0.2667
    * ROC AUC OVR (Test): 0.6463
    * Tiempo Entrenamiento Inicial (GridSearchCV CV=2): 155.23 segundos

**Elección:**
Se selecciona el modelo **KNN** como el modelo individual final.

**Razonamiento:**
Ambos modelos, KNN y SVC, mostraron **rendimientos idénticos** en términos de Accuracy (0.6522) y F1-Macro (0.2667) en el conjunto de prueba después del ajuste fino.

Considerando las métricas secundarias en el *test set*:
* El ROC AUC OVR fue ligeramente superior para SVC (0.6463) en comparación con KNN (0.6196), lo que sugiere una mejor capacidad de discriminación general por parte de SVC.

Sin embargo, en cuanto al **tiempo de entrenamiento** (basado en la búsqueda inicial con CV=2), el modelo **KNN fue significativamente más rápido** (9.06 segundos) en comparación con SVC (155.23 segundos). Esta diferencia de eficiencia es considerable.

Dado que ambos modelos tienen el mismo rendimiento en las métricas principales (Accuracy y F1-Macro) en el conjunto de prueba, y considerando la **gran ventaja en eficiencia computacional** del KNN, se elige **KNN** como el modelo final. Representa un mejor balance entre rendimiento y coste computacional para este problema bajo las condiciones evaluadas.